In [7]:
import os
workspace = '/Users/chip/dev/anyscale'
partition = 'train-00000-of-00738-80a58552f2fb3344-small'
outdir = f'{workspace}/{partition}'
imgdir = f'{outdir}/image'
os.makedirs(imgdir, exist_ok=True)
fnames = sorted([f for f in os.listdir(outdir) if f.endswith('html') and os.path.isfile(os.path.join(outdir, f))])

In [68]:
from bs4 import BeautifulSoup

def extract_text(html):
    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract all text from the parsed HTML
    text = soup.get_text()
    # Print the extracted text
    return ' '.join(text.split())

fname = fnames[0]
with open(os.path.join(outdir, fname), 'r') as f:
    html_content = f.read().strip()
    print(extract_text(html_content))


Fashion Brand Fashion Brand Fashion Brand is a leading fashion brand that offers a wide range of high-quality, stylish clothing and accessories. Our collections are designed to reflect the latest fashion trends and are made from high-quality materials. We are committed to providing our customers with the best possible shopping experience and we strive to make our collections accessible to all. Home About Contact


<!doctype html> <html lang="en"> <head>   <meta charset="utf-8">   <meta name="viewport" content="width=device-width, initial-scale=1.0">   <title>welcome to our technology company</title>   <script src="https://cdn.tailwindcss.com"></script>   <style>     body {       font-family: 'arial', sans-serif;       background-color: #000;       color: #fff;     }     .header {       background-color: #000;       color: #fff;       padding: 20px;       margin: 0;       border: 0;     }     .header h1 {       font-size: 36px;       margin: 0;     }     .header p {       font-size: 18px;       margin: 0;     }     .content {       background-color: #000;       color: #fff;       padding: 20px;       margin: 0;       border: 0;     }     .content p {       font-size: 18px;       margin: 0;     }     .footer {       background-color: #000;       color: #fff;       padding: 20px;       margin: 0;       border: 0;     }     .footer p {       font-size: 18px;       margin: 0;     }   </style> </head>

False
False
False
False
False
False
False
False
False
False


In [65]:
from html.parser import HTMLParser

class MyHTMLValidator(HTMLParser):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stack = []
        self.errors = []

    def handle_starttag(self, tag, attrs):
        self.stack.append(tag)

    def handle_endtag(self, tag):
        if not self.stack or self.stack[-1] != tag:
            self.errors.append(f"Unexpected closing tag: {tag} or missing closing tag for {self.stack[-1]}")
        else:
            self.stack.pop()

    def validate(self, html):
        self.feed(html)
        # Check for any unclosed tags
        if self.stack:
            self.errors.extend([f"Missing closing tag for {tag}" for tag in self.stack])
        return self.errors


def validate_html_tags(fnames):
    """Return the number HTML errors like missing closing tags.
    Note that even the original HTML code has a lot of missing closing tags"""
    validator = MyHTMLValidator()
    error_counts = []
    for fname in fnames:
        with open(os.path.join(outdir, fname), 'r') as f:
            html_content = f.read().strip()
            errors = validator.validate(html_content)
            error_counts.add(len(errors))
        return sum(error_counts) / len(error_counts)


In [61]:
gturl = '/Users/chip/dev/anyscale/data/eval-d2c/train-00000-of-00738-80a58552f2fb3344-1.html'
with open(os.path.join(outdir, gturl), 'r') as f:
    html_content = f.read().strip()
    errors = validator.validate(html_content)

    if errors:
        print("HTML Errors Found:", len(errors))
        for error in errors:
            print(error)
    else:
        print("No HTML errors found.")

HTML Errors Found: 24
Unexpected closing tag: head or missing closing tag for meta
Unexpected closing tag: div or missing closing tag for img
Unexpected closing tag: div or missing closing tag for img
Unexpected closing tag: body or missing closing tag for img
Unexpected closing tag: html or missing closing tag for img
Missing closing tag for html
Missing closing tag for head
Missing closing tag for meta
Missing closing tag for meta
Missing closing tag for body
Missing closing tag for div
Missing closing tag for div
Missing closing tag for img
Unexpected closing tag: html or missing closing tag for link
Missing closing tag for html
Missing closing tag for head
Missing closing tag for meta
Missing closing tag for meta
Missing closing tag for body
Missing closing tag for div
Missing closing tag for div
Missing closing tag for img
Missing closing tag for html
Missing closing tag for link


In [ ]:
from bs4 import BeautifulSoup
def is_proper_html(text):
    try:
        # Parse the text with BeautifulSoup using the "html.parser"
        soup = BeautifulSoup(text, "html.parser")
        # Generate the parsed text back to HTML string
        parsed_html = soup.prettify()
        # If parsing was successful and didn't result in an empty string, consider it proper HTML
        return bool(parsed_html.strip())
    except Exception as e:
        # If any parsing error occurs, it's not proper HTML
        return False

In [ ]:
from bs4 import BeautifulSoup

def has_basic_structure_and_closing_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    # Check for essential HTML structure: <html>, <head>, <title>, <body>
    if not soup.html or not soup.head or not soup.title or not soup.body:
        return False
    
    # Attempt to find obviously unclosed tags by converting the parsed structure back to string
    # and comparing it with the original HTML string.
    # This is a simplistic approach and might not catch nested or complex issues.
    rebuilt_html = str(soup)
    clean_rebuilt = rebuilt_html.strip().replace('\n', ' ').replace('  ', ' ').lower()
    clean_html = html.strip().replace('\n', ' ').replace('  ', ' ').lower()
    # print(rebuilt_html)
    # This comparison is very basic and might falsely flag correctly structured HTML
    # as incorrect if BeautifulSoup reformats the HTML, which it often does.
    print(clean_html[:1000])
    print("=" * 20)
    print(clean_rebuilt[:1000])
    return clean_rebuilt == clean_html

# Example HTML
html_text = "<html><head><title>Test Page</title></head><body><p>Sample Text</p></body></html>"

# Test the function
print(has_basic_structure_and_closing_tags(html_content))
# print(has_basic_structure_and_closing_tags(html_text))


In [ ]:
for fname in fnames:
    with open(os.path.join(outdir, fname), 'r') as f:
        html_content = f.read().strip()
        print(has_basic_structure_and_closing_tags(html_content))

In [ ]:
from lxml import html, etree

def is_well_formed_html(text):
    try:
        # Parse the text using lxml's HTML parser
        document = html.fromstring(text)
        # Serialize it back to a string
        serialized_document = etree.tostring(document, pretty_print=True, method="html").decode()
        # If no exception was raised during parsing, we consider it well-formed
        return True
    except etree.XMLSyntaxError as e:
        # If a parsing error occurs, it's not well-formed HTML
        return False

# Example usage:
html_text = "<ht><head><title>Valid Title</title></head><body><p>Hello World!</p>"
print(is_well_formed_html(html_text))
